# Arbitrage in the Crypto Exchange Market

In [187]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time

## Use https://www.coinapi.io/ APIs (trial version for now)

In [ ]:
API_Key = '87EF7C63-192A-4D5A-AF67-FCD8386A4168'

### Metadata

In [195]:
# List all periods

url = 'https://rest.coinapi.io/v1/ohlcv/periods'
headers = {'X-CoinAPI-Key' : API_Key}
all_periods = requests.get(url, headers=headers)

In [233]:
pd.DataFrame(all_periods.json())

,display_name,length_months,length_seconds,period_id,unit_count,unit_name
0,1 Second,0,1,1SEC,1,second
1,2 Seconds,0,2,2SEC,2,second
2,3 Seconds,0,3,3SEC,3,second
3,4 Seconds,0,4,4SEC,4,second
4,5 Seconds,0,5,5SEC,5,second
5,6 Seconds,0,6,6SEC,6,second
6,10 Seconds,0,10,10SEC,10,second
7,15 Seconds,0,15,15SEC,15,second
8,20 Seconds,0,20,20SEC,20,second
9,30 Seconds,0,30,30SEC,30,second


In [196]:
# List all symbols

url = 'https://rest.coinapi.io/v1/symbols'
headers = {'X-CoinAPI-Key' : API_Key'}
all_symbols = requests.get(url, headers=headers)

In [253]:
df = pd.DataFrame(all_symbols.json())[['symbol_id', 'asset_id_base','asset_id_quote']]

In [321]:
# selected by BTC/USDT pair

df.loc[df['asset_id_base']== 'BTC'].loc[df['asset_id_quote']== 'USDT']

,symbol_id,asset_id_base,asset_id_quote
12,BINANCE_SPOT_BTC_USDT,BTC,USDT
23,POLONIEX_SPOT_BTC_USDT,BTC,USDT
58,BITTREX_SPOT_BTC_USDT,BTC,USDT
70,HUOBIPRO_SPOT_BTC_USDT,BTC,USDT
108,HITBTC_SPOT_BTC_USDT,BTC,USDT
148,OKEX_SPOT_BTC_USDT,BTC,USDT
292,LIQUI_SPOT_BTC_USDT,BTC,USDT
410,CRYPTOPIA_SPOT_BTC_USDT,BTC,USDT
587,GATEIO_SPOT_BTC_USDT,BTC,USDT
1252,EXMO_SPOT_BTC_USDT,BTC,USDT


### I chose the top 2 cryptocurrency exchanges by volume: **Binance** & **OKEx**
ranked by https://coinmarketcap.com/rankings/exchanges/

### Get Binance historical price

In [237]:
url = 'https://rest.coinapi.io/v1/ohlcv/BINANCE_SPOT_BTC_USDT/history?period_id=1HRS&time_start=2018-10-20T00:00:00'
headers = {'X-CoinAPI-Key' : API_Key}
hist_data = requests.get(url, headers=headers)

In [309]:
binance_price_detail = pd.DataFrame(hist_data.json())

In [311]:
binance_price_detail['time_period_start'] = pd.to_datetime(binance_price_detail.time_period_start).dt.strftime('%Y/%m/%d %H')

In [312]:
binance_price_detail[['time_period_start','price_close']].head(5)

,time_period_start,price_close
0,2018/10/20 00,6518.06
1,2018/10/20 01,6545.23
2,2018/10/20 02,6537.20
3,2018/10/20 03,6543.05
4,2018/10/20 04,6549.11


In [313]:
binance_price = pd.Series(binance_price_detail['price_close'].values, index=okex_price_detail['time_period_start'])

### Get OKEx historical price

In [258]:
url = 'https://rest.coinapi.io/v1/ohlcv/OKEX_SPOT_BTC_USDT/history?period_id=1HRS&time_start=2018-10-20T00:00:00'
headers = {'X-CoinAPI-Key' : API_Key}
ok_hist_data = requests.get(url, headers=headers)

In [294]:
okex_price_detail = pd.DataFrame(ok_hist_data.json())

In [296]:
okex_price_detail['time_period_start'] = pd.to_datetime(okex_price_detail.time_period_start).dt.strftime('%Y/%m/%d %H')

In [298]:
okex_price_detail[['time_period_start','price_close']].head(5)

,time_period_start,price_close
0,2018/10/20 00,6515.0285
1,2018/10/20 01,6543.0000
2,2018/10/20 02,6536.1210
3,2018/10/20 03,6540.3929
4,2018/10/20 04,6544.2286


In [306]:
okex_price = pd.Series(okex_price_detail['price_close'].values, index=okex_price_detail['time_period_start'])

### Pick the data I need (**start time** and **close price**) and combine them into a pandas DataFrame.

In [ ]:
dictionary = {
    'binance':binance_price,
    'okex':okex_price
}

price = pd.DataFrame(dictionary)

borrows on Binance and lends on OKEx<br>
profitable if ratio > 0 (neglect transaction fee)

In [319]:
price['ratio'] = price.okex / price.binance

In [320]:
price

,binance,okex,ratio
time_period_start,,,
2018/10/20 00,6518.06,6515.0285,0.999535
2018/10/20 01,6545.23,6543.0000,0.999659
2018/10/20 02,6537.20,6536.1210,0.999835
2018/10/20 03,6543.05,6540.3929,0.999594
2018/10/20 04,6549.11,6544.2286,0.999255
2018/10/20 05,6551.48,6547.5900,0.999406
2018/10/20 06,6591.00,6591.9505,1.000144
2018/10/20 07,6576.43,6574.2838,0.999674
2018/10/20 08,6573.97,6573.9966,1.000004
